In [71]:
import pandas as pd
import json
import requests

In [84]:
# Ajoute à la vairable data la réponse json de l'appel à l'API https://api.gouv.fr/documentation/api_etablissements_publics
# avec le mot-clé "permanence_juridique"

url = "https://etablissements-publics.api.gouv.fr/v3/organismes/permanence_juridique" 
response = requests.get(url)

data = response.json()

In [87]:
# ATTENTION : il manque les horaires

# Initialisation des dictionnaires
myAll = {}
myDict = {}
myAddresses = {}


# La boucle va chercher pour chaque occurence dans le json l'identifiant qui sera une key dans le dict myAll()
# Ensuite elle va chercher toutes les properties pour les mettre dans un dictonnaire "temporaire" myDict()
# Toutes les properties contenu dans myDict() sont enfin ajoutée au dictionnaire myAll() avec pour key l'id de la permanance
# PS : désolé pour les try : except : car ça pique un peu les yeux

for x in data['features'][0]:
    myDict = {}
    identifiant = x['properties']['id']


    myDict['coordinates'] = x['geometry']['coordinates']
    myDict['codeInsee'] = x['properties']['codeInsee']
    myDict['nom'] = x['properties']['nom']
    
    for a in x['properties']['adresses']:
        myDict['adresse_ligne01'] = a['lignes'][0]
        try :
            myDict['adresse_ligne02'] = a['lignes'][1]
        except: IndexError 
        
        try:
            myDict['adresse_ligne03'] = a['lignes'][2]
        except: IndexError
        
        myDict['adresse_code_postal'] = a['codePostal']
        myDict['adresse_commune'] = a['commune']
        myDict['adresse_coordonnees'] = a['coordonnees']
        
    try :
        myDict['e_mail'] = x['properties']['email']
    except : 
        pass
    
    try :
        myDict['telephone'] = x['properties']['telephone']
    except : 
        pass

    try :
        myDict['url'] = x['properties']['url']
    except :
        pass
    
    try :
        myDict['zonage_commune'] = x['properties']['zonage']['communes']
    except :
        pass
    
    try: 
        myDict['zonage_commune'] = x['properties']['zonage']['communes']
    except: 
        pass
    
    myAll[identifiant] = myDict

In [88]:
#Transforme le dictionnaire myAll() en dataframe puis réorganise l'ordre des colonnes et l'index

df = pd.DataFrame.from_dict(myAll).transpose()
df = df.reindex(columns = ['coordinates', 'codeInsee', 'nom', 'adresse_ligne01', 'adresse_ligne02', 'adresse_ligne03', 'adresse_code_postal', 'adresse_commune', 'adresse_coordonnees', 'telephone', 'e_mail', 'url', 'zonage_commune'])
df = df.reset_index()

In [90]:
#Export le dataframe en csv

df.to_csv('~/scripts/jupyter/permanence_juridique/2020-02-08_permanence_juridique.csv')